4) enpoint:
- def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
- Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [3]:
import pandas as pd
games = pd.read_parquet('../Dataset/games_clean.parquet')
reviews = pd.read_parquet('../Dataset/reviews_clean.parquet')

In [4]:
games.columns

Index(['id', 'title', 'developer', 'release_date', 'genres', 'tags', 'price'], dtype='object')

In [5]:
reviews.columns

Index(['item_id', 'negativo', 'neutral', 'positivo'], dtype='object')

In [6]:
#Creamos dos dataframes donde contengan las columnas que vamos a utilizar 
games = games[['developer', 'title', 'id', 'genres', 'release_date']]
reviews = reviews[['item_id', 'positivo']]

In [7]:
games.head(5)

,developer,title,id,genres,release_date
0,Kotoshiro,Lost Summoner Kitty,761140,"Action, Casual, Indie, Simulation, Strategy",2018-01-04
1,Secret Level SRL,Ironbound,643980,"Free to Play, Indie, RPG, Strategy",2018-01-04
2,Poolians.com,Real Pool 3D - Poolians,670290,"Casual, Free to Play, Indie, Simulation, Sports",2017-07-24
3,彼岸领域,弹炸人2222,767400,"Action, Adventure, Casual",2017-12-07
4,Trickjump Games Ltd,Battle Royale Trainer,772540,"Action, Adventure, Simulation",2018-01-04


In [8]:
reviews.head(5)

,item_id,positivo
0,10,51
1,10090,47
2,10130,2
3,10140,0
4,10150,9


In [9]:
#Combertimos el archivo a parquet
games.to_parquet('../Dataset/best_developer_games.parquet', compression='snappy', index=False)
reviews.to_parquet('../Dataset/best_developer_reviews.parquet', compression='snappy', index=False)

-------------------------------------------------------------------------------------------------------------------------

Probando el endpoint

In [16]:
def best_developer_year(año: int):
    """
    4) enpoint:
    - def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
    - Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    """
    global games, reviews
    games['release_year'] = games['release_date'].dt.year
    games_filtered = games[games['release_year'] == año]
    filtered_reviews = games_filtered.merge(reviews, left_on='id', right_on='item_id', how='inner')
    filtered_reviews = filtered_reviews[['developer', 'title', 'positivo']]
    filtered_reviews = filtered_reviews.groupby(['developer']).agg({'positivo': 'sum'}).reset_index()
    filtered_reviews = filtered_reviews.sort_values(by='positivo', ascending=False)

    top_developers = filtered_reviews.head(3)['developer'].tolist()
    return [
        {"Puesto 1": top_developers[0]},
        {"Puesto 2": top_developers[1]},
        {"Puesto 3": top_developers[2]}
    ]

prueba = best_developer_year(2000)
print(prueba)

[{'Puesto 1': 'Valve'}, {'Puesto 2': 'Ion Storm'}, {'Puesto 3': 'Rebellion'}]
